In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
import time  # To time the tuning process
from pytorch_model import PyTorchRegressorWrapper
from sklearn.inspection import permutation_importance
import warnings

In [2]:
df_train = pd.read_csv('C://Users//vince//PycharmProjects//Data Mining//Used-Car-Data-Mining//train_set.csv')
df_test = pd.read_csv('C://Users//vince//PycharmProjects//Data Mining//Used-Car-Data-Mining//test_set.csv')

In [3]:
len(df_train)

341504

In [4]:
df_train.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7310804917,https://austin.craigslist.org/ctd/d/houston-20...,austin,https://austin.craigslist.org,46995,2017.0,ford,f-150,NaN,8 cylinders,...,NaN,truck,black,https://images.craigslist.org/00202_fA8s718RZU...,This 2017 Ford F-150 Lariat 4WD SuperCrew 5.5'...,NaN,tx,29.678380,-95.435120,2021-04-22T09:03:58-0500
1,7315327041,https://sandiego.craigslist.org/nsd/ctd/d/palm...,san diego,https://sandiego.craigslist.org,0,2018.0,acura,mdx,NaN,6 cylinders,...,NaN,SUV,NaN,https://images.craigslist.org/00Q0Q_b7u8uPQBQE...,Coachella Valley Volkswagen STOCK #: P...,NaN,ca,33.755638,-116.286258,2021-05-01T08:55:38-0700
2,7310280162,https://albany.craigslist.org/ctd/d/new-lebano...,albany,https://albany.craigslist.org,36900,2016.0,ford,f-250 super duty,excellent,8 cylinders,...,full-size,truck,white,https://images.craigslist.org/00c0c_jcqfcYXhpQ...,2016 Ford F-250 Super Duty XL 4x4 4dr Crew Cab...,NaN,ny,42.475900,-73.377300,2021-04-21T09:39:24-0400
3,7315777281,https://washingtondc.craigslist.org/nva/ctd/d/...,"washington, DC",https://washingtondc.craigslist.org,1399,2021.0,NaN,WOLF ISLANDER,NaN,NaN,...,NaN,other,orange,https://images.craigslist.org/00000_af225n2gi4...,2021 WOLF ISLANDER 50CC Offered by: Beyond...,NaN,dc,38.759303,-77.454234,2021-05-02T12:05:03-0400
4,7306277173,https://york.craigslist.org/ctd/d/york-2018-bu...,york,https://york.craigslist.org,29590,2018.0,buick,enclave essence sport,good,6 cylinders,...,NaN,SUV,white,https://images.craigslist.org/00w0w_36PXCHco1K...,Carvana is the safer way to buy a car During t...,NaN,pa,39.960000,-76.730000,2021-04-13T14:21:01-0400


In [5]:
coloums_to_drop = ['url', 'county', 'lat', 'long', 'region_url', 'posting_date', 'VIN', 'image_url', 'description', 'id']
df_train.drop(columns=coloums_to_drop, inplace=True)

In [6]:
df_train.head()

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,austin,46995,2017.0,ford,f-150,NaN,8 cylinders,gas,48544.0,clean,automatic,4wd,NaN,truck,black,tx
1,san diego,0,2018.0,acura,mdx,NaN,6 cylinders,gas,53103.0,clean,automatic,fwd,NaN,SUV,NaN,ca
2,albany,36900,2016.0,ford,f-250 super duty,excellent,8 cylinders,gas,73394.0,clean,automatic,4wd,full-size,truck,white,ny
3,"washington, DC",1399,2021.0,NaN,WOLF ISLANDER,NaN,NaN,gas,500.0,clean,automatic,NaN,NaN,other,orange,dc
4,york,29590,2018.0,buick,enclave essence sport,good,6 cylinders,gas,38459.0,clean,other,fwd,NaN,SUV,white,pa


In [8]:
df_train['cylinders'] = df_train['cylinders'].str.split(' ', expand=True,regex=True)[0].astype(int)

ValueError: cannot convert float NaN to integer

In [7]:
df_train.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7310804917,https://austin.craigslist.org/ctd/d/houston-20...,austin,https://austin.craigslist.org,46995,2017.0,ford,f-150,NaN,8,...,NaN,truck,black,https://images.craigslist.org/00202_fA8s718RZU...,This 2017 Ford F-150 Lariat 4WD SuperCrew 5.5'...,NaN,tx,29.678380,-95.435120,2021-04-22T09:03:58-0500
1,7315327041,https://sandiego.craigslist.org/nsd/ctd/d/palm...,san diego,https://sandiego.craigslist.org,0,2018.0,acura,mdx,NaN,6,...,NaN,SUV,NaN,https://images.craigslist.org/00Q0Q_b7u8uPQBQE...,Coachella Valley Volkswagen STOCK #: P...,NaN,ca,33.755638,-116.286258,2021-05-01T08:55:38-0700
2,7310280162,https://albany.craigslist.org/ctd/d/new-lebano...,albany,https://albany.craigslist.org,36900,2016.0,ford,f-250 super duty,excellent,8,...,full-size,truck,white,https://images.craigslist.org/00c0c_jcqfcYXhpQ...,2016 Ford F-250 Super Duty XL 4x4 4dr Crew Cab...,NaN,ny,42.475900,-73.377300,2021-04-21T09:39:24-0400
3,7315777281,https://washingtondc.craigslist.org/nva/ctd/d/...,"washington, DC",https://washingtondc.craigslist.org,1399,2021.0,NaN,WOLF ISLANDER,NaN,NaN,...,NaN,other,orange,https://images.craigslist.org/00000_af225n2gi4...,2021 WOLF ISLANDER 50CC Offered by: Beyond...,NaN,dc,38.759303,-77.454234,2021-05-02T12:05:03-0400
4,7306277173,https://york.craigslist.org/ctd/d/york-2018-bu...,york,https://york.craigslist.org,29590,2018.0,buick,enclave essence sport,good,6,...,NaN,SUV,white,https://images.craigslist.org/00w0w_36PXCHco1K...,Carvana is the safer way to buy a car During t...,NaN,pa,39.960000,-76.730000,2021-04-13T14:21:01-0400


In [6]:
df_train['cylinders'].unique()

array(['8', '6', nan, '4', '5', 'other', '3', '10', '12'], dtype=object)

In [7]:
df_train.nunique()

region            404
price           14350
year              112
manufacturer       42
model           26401
condition           6
cylinders           8
fuel                5
odometer        93931
title_status        6
transmission        3
drive               3
size                4
type               13
paint_color        12
state              51
dtype: int64

In [8]:
df_train['title_status'].unique()

array(['clean', nan, 'rebuilt', 'salvage', 'lien', 'missing',
       'parts only'], dtype=object)

In [9]:
X_train = df_train.drop(columns=['price'])
y_train = df_train['price']
X_test = df_test.drop(columns=['price'])
y_test = df_test['price']

In [10]:

from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, PolynomialFeatures
import category_encoders as ce
from sklearn.compose import ColumnTransformer
from pygam import LinearGAM

# Define columns
cat_columns = ['cylinders', 'region', 'manufacturer', 'state', 'model', 'fuel', 'drive', 'transmission', 'type', 'condition', 'size', 'title_status', 'paint_color' ]
num_columns = ['year', 'odometer']

# Create proper preprocessing pipeline using ColumnTransformer
preprocessor = ColumnTransformer([
    # For categorical features, split by encoding typ
    ('region_enc', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', ce.BinaryEncoder())
    ]), ['region']),
    
    ('freq_enc', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', ce.CountEncoder())
    ]), ['manufacturer', 'state', 'model']),
    
    ('onehot_enc', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ]), ['fuel', 'drive', 'transmission', 'type', 'title_status', 'paint_color', 'cylinders']),
    
    ('ordinal_enc', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OrdinalEncoder())
    ]), ['condition', 'size']),
    
    # For numerical features
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]), num_columns)
], remainder='drop')


# Dictionary of regressors
model_pipelines = {
    # --- Linear Models ---
    'linear_simple': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression(n_jobs=-1),)
    ]),
    'linear_poly': Pipeline([
        ('preprocessor', preprocessor),
        ('poly_features', PolynomialFeatures(include_bias=False)), # Degree will be tuned
        ('regressor', LinearRegression(n_jobs=-1))
    ]),
    'ridge': Pipeline([ # NEW: Simple Ridge
        ('preprocessor', preprocessor),
        ('regressor', Ridge(random_state=42)) # alpha will be tuned
    ]),
    'ridge_poly': Pipeline([ # NEW: Polynomial Ridge
        ('preprocessor', preprocessor),
        ('poly_features', PolynomialFeatures(include_bias=False)), # Degree will be tuned
        ('regressor', Ridge(random_state=42)) # alpha will be tuned
    ]),
    'lasso': Pipeline([ # NEW: Simple Lasso
        ('preprocessor', preprocessor),
        ('regressor', Lasso(random_state=42)) # alpha will be tuned
    ]),
    'lasso_poly': Pipeline([ # NEW: Polynomial Lasso
        ('preprocessor', preprocessor),
        ('poly_features', PolynomialFeatures(include_bias=False)), # Degree will be tuned
        ('regressor', Lasso(random_state=42)) # alpha will be tuned
    ]),
    'gam': Pipeline([ # NEW: Generalized Additive Models
        ('preprocessor', preprocessor),
        ('regressor', LinearGAM(random_state=42))
    ]),
    'svr': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', SVR()) # Note: SVR is computationally expensive, esp. with many features (like from poly)
    ]),
    'dt': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', DecisionTreeRegressor(random_state=42))
    ]),
    'rf': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(random_state=42, n_jobs=-1))
    ]),
    'gb': Pipeline([ # NEW: gradBoost
        ('preprocessor', preprocessor),
        ('regressor', HistGradientBoostingRegressor(random_state=42))
    ]),
    'xgb': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', xgb.XGBRegressor(random_state=42, n_jobs=-1, objective='reg:squarederror'))
    ])
    'pytorch_linear': Pipeline([
        ('preprocessor', preprocessor), # Use the SAME preprocessor!
        ('regressor', PyTorchRegressorWrapper(lr=0.01, epochs=50, batch_size=64)) # Adjust hyperparameters
    ])
}

In [11]:
import numpy as np
from scipy.stats import loguniform, uniform, randint
from sklearn.model_selection import RandomizedSearchCV
# Define parameter distributions for RandomizedSearchCV
param_distributions = {
    # --- Linear Models ---
    'linear_simple': {
        'regressor__fit_intercept': [True, False],
        'regressor__positive': [True, False]
    },
     'linear_poly': {
        'poly_features__degree': [2, 3], # Tune polynomial degree
        'regressor__fit_intercept': [True, False],
        'regressor__positive': [True, False]
    },
    'ridge': { # NEW
        'regressor__alpha': loguniform(1e-3, 1e2), # Regularization strength (e.g., 0.001 to 100)
        'regressor__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'] # Optional: Different solvers
    },
    'ridge_poly': { # NEW
        'poly_features__degree': [2, 3],
        'regressor__alpha': loguniform(1e-3, 1e2),
        'regressor__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
    },
    'lasso': { # NEW
        'regressor__alpha': loguniform(1e-4, 1e1), # Regularization strength (e.g., 0.0001 to 10) - Lasso often needs smaller alpha
        'regressor__max_iter': randint(1000, 5000) # Lasso might need more iterations to converge
    },
    'lasso_poly': { # NEW
        'poly_features__degree': [2, 3],
        'regressor__alpha': loguniform(1e-4, 1e1),
        'regressor__max_iter': randint(1000, 5000)
    },
    'gam': {
        'regressor__n_splines': randint(5, 25),
        'regressor__lam': loguniform(1e-3, 1e2),
    },
    'svr': {
        'regressor__C': loguniform(1e-1, 1e2),
        'regressor__epsilon': loguniform(1e-2, 1e0),
        'regressor__gamma': loguniform(1e-4, 1e-1),
        'regressor__kernel': ['linear', 'rbf']
    },
    'dt': {
        'regressor__max_depth': [None] + list(range(5, 21, 3)),
        'regressor__min_samples_split': randint(2, 11),
        'regressor__min_samples_leaf': randint(1, 11),
        'regressor__max_features': ['sqrt', 'log2', 0.7, 0.9]
    },
    'rf': {
        'regressor__n_estimators': randint(50, 251),
        'regressor__max_depth': [None] + list(range(10, 31, 5)),
        'regressor__min_samples_split': randint(2, 7),
        'regressor__min_samples_leaf': randint(1, 5),
        'regressor__max_features': ['sqrt', 0.6, 0.8],
        'regressor__bootstrap': [True]
    },
    'gb': {
        'regressor__n_estimators': randint(50, 201),
        'regressor__learning_rate': loguniform(0.01, 0.2),
        'regressor__max_depth': randint(3, 9),
        'regressor__min_samples_split': randint(2, 7),
        'regressor__min_samples_leaf': randint(1, 5),
        'regressor__max_features': ['sqrt', 0.6, 0.8],
        'regressor__subsample': uniform(0.6, 0.4),  
        'regressor__ccp_alpha': loguniform(1e-3, 1.0)
    },
    'xgb': {
        'regressor__n_estimators': randint(50, 301),
        'regressor__learning_rate': loguniform(0.01, 0.2),
        'regressor__max_depth': randint(3, 9),
        'regressor__min_child_weight': randint(1, 7),
        'regressor__subsample': uniform(0.6, 0.4),
        'regressor__colsample_bytree': uniform(0.6, 0.4),
        'regressor__gamma': loguniform(1e-2, 1.0),
        'regressor__reg_alpha': loguniform(1e-3, 1.0), # Note: XGBoost also has alpha/lambda params
        'regressor__reg_lambda': loguniform(1e-3, 10.0)
    },
    'pytorch_linear': {
        'regressor__lr': loguniform(1e-4, 1e-1), # Tune learning rate
        'regressor__epochs': randint(20, 101),   # Tune number of epochs
        'regressor__batch_size': [32, 64, 128]   # Tune batch size
    }
}
print("Hyperparameter distributions updated for Ridge and Lasso.")

In [12]:
def tune_model(X_train, y_train, model_name, pipeline, param_dist, cv=5, n_iter=10):
    """
    Performs RandomizedSearchCV for a given model pipeline.

    Args:
        X_train: Training features.
        y_train: Training target.
        model_name (str): Name of the model for printing.
        pipeline (Pipeline): The scikit-learn pipeline (preprocessor + regressor).
        param_dist (dict): Parameter distribution for the regressor.
        cv (int): Number of cross-validation folds.
        n_iter (int): Number of parameter settings sampled.

    Returns:
        best_estimator: The best model pipeline found by RandomizedSearchCV.
    """
    print(f"--- Tuning {model_name} ---")
    start_time = time.time()

    # Create RandomizedSearchCV object
    random_search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_dist,
        n_iter=n_iter,                     # Number of iterations (samples from search space)
        cv=cv,                             # Cross-validation folds
        scoring='neg_mean_squared_error',  # Metric to optimize (negative MSE, higher is better)
        n_jobs=1,                          # Use 1 core for sequential processing as requested (can set to -1 for parallel)
        verbose=1,                         # Show progress updates
        random_state=42,                   # For reproducibility
        return_train_score=True            # Useful for diagnosing overfitting/underfitting
    )

    # Fit the random search to the training data
    random_search.fit(X_train, y_train)

    end_time = time.time()
    elapsed_time = end_time - start_time

    # Print tuning results
    print(f"\nFinished tuning {model_name} in {elapsed_time:.2f} seconds.")
    print(f"Best parameters found:")
    # Display parameters without the 'regressor__' prefix for clarity
    best_params_cleaned = {k.replace('regressor__', ''): v for k, v in random_search.best_params_.items()}
    print(best_params_cleaned)
    # The score is negative MSE, so we negate it to show positive MSE
    best_cv_mse = -random_search.best_score_
    best_cv_rmse = np.sqrt(best_cv_mse)
    print(f"Best cross-validation score (MSE): {best_cv_mse:.4f}")
    print(f"Best cross-validation score (RMSE): {best_cv_rmse:.4f}")
    print("-------------------------")

    return random_search.best_estimator_


In [ ]:
n_iterations = 10 # Set the number of iterations for Randomized Search
cv_folds = 3      # Use 3-fold CV for faster tuning, 5 is also common

best_models = {} # Dictionary to store the best tuned model for each type

for model_name, pipeline in model_pipelines.items():
    if model_name in param_distributions:
        best_models[model_name] = tune_model(
            X_train, y_train, model_name, pipeline,
            param_distributions[model_name], cv=cv_folds, n_iter=n_iterations
        )
    else:
        print(f"--- Training {model_name} (no tuning parameters defined) ---")
        start_time = time.time()
        pipeline.fit(X_train, y_train)
        best_models[model_name] = pipeline # Store the fitted pipeline
        end_time = time.time()
        print(f"Finished training {model_name} in {end_time - start_time:.2f} seconds.")
        print("-------------------------")


--- Tuning linear ---
Fitting 3 folds for each of 4 candidates, totalling 12 fits


C:\Users\vince\PycharmProjects\Data Mining\.venv\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(



Finished tuning linear in 135.69 seconds.
Best parameters found:
{'positive': True, 'fit_intercept': False}
Best cross-validation score (MSE): 87671198223543.5156
Best cross-validation score (RMSE): 9363289.9252
-------------------------
--- Tuning svr ---
Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [14]:
from sklearn.metrics import mean_squared_error, r2_score

print("\n--- Test Set Evaluation ---")
results = []

for name, model in best_models.items():
    print(f"Evaluating {name}...")
    start_time = time.time()

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate evaluation metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    end_time = time.time()
    eval_time = end_time - start_time

    # Store results
    results.append({
        'Model': name,
        'Test MSE': mse,
        'Test RMSE': rmse,
        'Test R2 Score': r2,
        'Evaluation Time (s)': eval_time
    })

    # Print results for the current model
    print(f"  Test MSE: {mse:.4f}")
    print(f"  Test RMSE: {rmse:.4f}")
    print(f"  Test R² Score: {r2:.4f}")
    print(f"  Evaluation Time: {eval_time:.2f} seconds")
    print("-" * 20)
    
# Convert results to a DataFrame for better visualization and saving
results_df = pd.DataFrame(results)
print("\n--- Overall Test Set Results ---")
print(results_df.to_string(index=False)) # .to_string() prints the full df nicely



--- Test Set Evaluation ---

--- Overall Test Set Results ---
Empty DataFrame
Columns: []
Index: []


In [ ]:
# Define the path for the results CSV file
results_csv_path = 'model_evaluation_results.csv'

try:
    # Save the DataFrame to a CSV file
    results_df.to_csv(results_csv_path, index=False)
    print(f"\nEvaluation results saved successfully to: {results_csv_path}")
except Exception as e:
    print(f"\nError saving results to CSV: {e}")

In [ ]:
# Select a model to inspect (e.g., the best RandomForest)
model_to_inspect_name = 'rf' # Or 'xgb', 'linear_simple', etc.
if model_to_inspect_name in best_models:
    fitted_pipeline = best_models[model_to_inspect_name]
    print(f"Calculating permutation importance for: {model_to_inspect_name}")

    start_time = time.time()
    # Calculate importance on the TEST set
    perm_importance = permutation_importance(
        fitted_pipeline,
        X_test,
        y_test,
        n_repeats=10,       # Number of times to permute a feature
        random_state=42,
        n_jobs=1            # Can set to -1 for parallelism if desired
    )
    end_time = time.time()
    print(f"Calculation time: {end_time - start_time:.2f} seconds")

    # --- Get Feature Names ---
    # This is crucial but can be tricky with ColumnTransformer
    try:
        # Access the fitted preprocessor step
        preprocessor_step = fitted_pipeline.named_steps['preprocessor']
        # Get feature names out - this is the preferred way for newer scikit-learn
        feature_names = preprocessor_step.get_feature_names_out()
        print(f"Successfully retrieved {len(feature_names)} feature names.")

        # --- Display Results ---
        sorted_idx = perm_importance.importances_mean.argsort()[::-1] # Sort descending

        print(f"\nTop 15 Features for {model_to_inspect_name} (Permutation Importance):")
        for i in sorted_idx[:15]: # Show top 15
            print(f"{feature_names[i]:<30} "
                  f"Mean Importance: {perm_importance.importances_mean[i]:.4f} "
                  f" +/- {perm_importance.importances_std[i]:.4f}")

    except AttributeError:
         print("Warning: Could not automatically retrieve feature names using get_feature_names_out().")
         print("Feature importances calculated, but names are not available.")
         # You might need a more manual way depending on older scikit-learn versions
         # or specific transformer behavior.
         sorted_idx = perm_importance.importances_mean.argsort()[::-1]
         print(f"\nTop 15 Features (indices) for {model_to_inspect_name}:")
         for i in sorted_idx[:15]:
             print(f"Feature Index: {i:<5} "
                   f"Mean Importance: {perm_importance.importances_mean[i]:.4f} "
                   f" +/- {perm_importance.importances_std[i]:.4f}")
    except Exception as e:
        print(f"An error occurred during feature name retrieval or importance display: {e}")

else:
    print(f"Model '{model_to_inspect_name}' not found in best_models.")

print("-" * 30)